In [ ]:
import requests
import pandas as pd
import numpy as np
import math
from bs4 import BeautifulSoup
from datetime import datetime

In [27]:
#coletando os dados em todas as páginas (aula de páginação)
#teste multiple

#link princial do modelo de roupa que pŕeciso
url= 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

page = requests.get( url, headers=headers )


soup= BeautifulSoup(page.text,'html.parser')


#coletando o total de itens de todas as páginas
total_item= soup.find_all('h2',class_='load-more-heading')[0].get('data-total')

#calculando o número de paginas que preciso calcular para pesquisar pelas vitrines da loja
page_number =math.ceil(int(total_item)/36)

page_number

#montando url para pegar todos os produtos da página
url2= url +'?page-size=' + str(int(page_number *36))
url2

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

In [41]:
#montando as colunas que preciso para criar meu Data Frame
url = url2

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup(page.text, 'html.parser')

products = soup.find( 'ul', class_='products-listing small' )

products_list= products.find_all ('article',class_="hm-product-item")

#product id

product_id = [p.get('data-articlecode') for p in products_list]

#product category

products_category= [p.get('data-category') for p in products_list]

#product name 

#extraction all the name 
products_list_2=  products.find_all('a', class_='link')

products_name =[p.get_text('link') for p in products_list_2]

#product price

#extraction informations of price
products_list_3 = products.find_all('span', class_='price regular')

#find all the price in page
products_price= [p.get_text('price_regular').replace('$ ','') for p in products_list_3]

data = pd.DataFrame([product_id,products_category,products_name,products_price]).T

data.columns = ['product_id','products_category','products_name','products_price']

data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d-%H-%M:%S')

In [42]:
#quantidade por vitrine
print(len(data))

display(data)



98


,product_id,products_category,products_name,products_price,scrapy_datetime
0,0985159001,men_jeans_skinny,Skinny Jeans,19.99,2022-04-03-16-51:29
1,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-03-16-51:29
2,1004199005,men_jeans_skinny,Skinny Cropped Jeans,29.99,2022-04-03-16-51:29
3,0690449051,men_jeans_ripped,Skinny Jeans,39.99,2022-04-03-16-51:29
4,1024256002,men_jeans_slim,Slim Jeans,19.99,2022-04-03-16-51:29
...,...,...,...,...,...
93,0974202002,men_jeans_regular,Regular Denim Joggers,29.99,2022-04-03-16-51:29
94,0875105009,men_jeans_relaxed,Relaxed Jeans,39.99,2022-04-03-16-51:29
95,1018704002,men_jeans_relaxed,Relaxed Pull-on Jeans,24.99,2022-04-03-16-51:29
96,0993887004,men_jeans_joggers,Hybrid Regular Denim Joggers,39.99,2022-04-03-16-51:29


In [91]:
#key browser
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

#empty Data Frame (to insert the joins)
df_details= pd.DataFrame()

aux= []

cols= ['Composition', 'Fit', 'color_id', 'color_name', 'product_id', 'stlye_id']

df_pattern = pd.DataFrame( columns=cols )


#API Requests
for i in range(len(data)):
    
    url= 'https://www2.hm.com/en_us/productpage.'+ data.loc[i,'product_id'] +'.html'

    page = requests.get( url, headers=headers )

    #Beautifull Souo object
    soup= BeautifulSoup(page.text,'html.parser')

    ### ===================== color name ======================

    products_list= soup.find_all('a',class_='filter-option miniature')

    #product id
    products_id = [p.get('data-articlecode') for p in products_list]

    #color name
    color_name = [p.get('data-color') for p in products_list]

    #create data frame

    df_color = pd.DataFrame([products_id,color_name]).T

    #rename the columns of DF
    df_color.columns= ['product_id','color_name']

    #create new colum styly_id
    df_color['stlye_id'] = df_color['product_id'].apply(lambda x: x[:-3])

    #create new colum color_id
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

    # ============================== composition =============================== #

    #composition
    composition_list= soup.find_all('div', class_ = 'details-attributes-list-item')

    product_composition = [list(filter (None, p.get_text().split('\n') ) ) for p in composition_list]
    
    if product_composition != []:
        
        #created the dataframe aux
        df_aux= pd.DataFrame(product_composition).T
        #renome the columns
        df_aux.columns= df_aux.iloc[0]
        
        #select columns 
        df_aux= df_aux[['Fit','Composition','Art. No.']]

        #delete the first row and delete the columns that is the empty
        df_aux= df_aux.iloc[1:3]

        #copying the code and the model to row the below
        df_aux = df_aux.fillna(method='ffill')

        #rename the data frame to a new variable
        df_composition = df_aux.copy(True)
        
        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        #create new colum styly_id
        df_composition['stlye_id'] =df_composition['Art. No.'].apply(lambda x: x[:-3])

        #create new colum color_id
        df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
        
        aux= aux + df_composition.columns.tolist()
        
        #merge dataframe of color and composition 
        data_sku= pd.merge(df_color,df_composition[['stlye_id','Fit','Composition']], how='left', on ='stlye_id')

        #
        df_details= pd.concat([df_details,data_sku], axis=0)
        
    else:
        None
# Join Showroom data + details

data['stlye_id'] = data['product_id'].apply( lambda x: x[:-3] )
data['color_id'] = data['product_id'].apply( lambda x: x[-3:] )


data_raw= pd.merge(data,df_details[['stlye_id','color_name','Fit','Composition']], how='left', on ='stlye_id')

In [95]:
data_raw.head()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Fit,Composition
0,0985159001,men_jeans_skinny,Skinny Jeans,19.99,2022-04-03-16-51:29,0985159,001,Denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
1,0985159001,men_jeans_skinny,Skinny Jeans,19.99,2022-04-03-16-51:29,0985159,001,Denim blue,Skinny fit,Pocket lining: Cotton 100%
2,0985159001,men_jeans_skinny,Skinny Jeans,19.99,2022-04-03-16-51:29,0985159,001,Dark gray,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
3,0985159001,men_jeans_skinny,Skinny Jeans,19.99,2022-04-03-16-51:29,0985159,001,Dark gray,Skinny fit,Pocket lining: Cotton 100%
4,0985159001,men_jeans_skinny,Skinny Jeans,19.99,2022-04-03-16-51:29,0985159,001,Light denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
